<a href="https://colab.research.google.com/github/bogdanjianu150/MaskRCNNv2/blob/main/p2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!git clone "https://github.com/SriRamGovardhanam/wastedata-Mask_RCNN-multiple-classes.git"

Cloning into 'wastedata-Mask_RCNN-multiple-classes'...
remote: Enumerating objects: 151, done.
remote: Total 151 (delta 0), reused 0 (delta 0), pack-reused 151
Receiving objects: 100% (151/151), 12.19 MiB | 17.29 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [2]:
import shutil, os
def copytree(src = '/content/wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN', dst = '/content/', symlinks=False, ignore=None):
    try:
      shutil.rmtree('/content/.ipynb_checkpoints')
    except:
      pass
    for item in os.listdir(src):
      s = os.path.join(src, item)
      d = os.path.join(dst, item)
      if os.path.isdir(s):
        shutil.copytree(s, d, symlinks, ignore)
      else:
        shutil.copy2(s, d)
copytree()

In [4]:
!pip install tensorflow==1.15.0
!pip install keras==2.2.5
!pip install h5py==2.10.0
!pip install -U scikit-image==0.16.2

     |████████████████████████████████| 412.3 MB 26 kB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
     |████████████████████████████████| 503 kB 43.4 MB/s 
     |████████████████████████████████| 3.8 MB 46.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=bb7f23470f4d431c6b984bbfc303f5cafcea28bdcec2cc99f329f2af28bbf977
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succes

     |████████████████████████████████| 336 kB 8.3 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0


     |████████████████████████████████| 2.9 MB 7.4 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


     |████████████████████████████████| 26.5 MB 1.4 MB/s 
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.3
    Uninstalling scikit-image-0.18.3:
      Successfully uninstalled scikit-image-0.18.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [1]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import re
import matplotlib.pyplot as plt
# Root directory of the project
ROOT_DIR = os.path.abspath("/content/drive/MyDrive/Mask_RCNN")
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.visualize import display_instances
from mrcnn.config import Config
from mrcnn import model as modellib, utils
# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join("/content/drive/MyDrive/mask_rcnn_coco.h5")
# Directory to save logs and model checkpoints
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
class CustomConfig(Config):
    """Configuration for training on the dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"
    
    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2
    # Number of classes (including background)
    NUM_CLASSES = 7  # Background + (Horse and Man)
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

Using TensorFlow backend.


In [10]:
dataset_path = os.path.abspath("/content/drive/MyDrive/trashnet")
images_path = os.path.abspath("/content/drive/MyDrive/trashnet/data_tv")
#images_path = os.path.abspath("/content/drive/MyDrive/trashnet/dataset")

masks_path = os.path.sep.join([dataset_path, "masks_modif.json"])

In [9]:
class CustomDataset(utils.Dataset):
  def load_custom(self, dataset, subset):
        """Load a subset of the Horse-Man dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("object", 1, "cardboard")
        self.add_class("object", 2, "glass")
        self.add_class("object", 3, "paper")
        self.add_class("object", 4, "plastic")
        self.add_class("object", 5, "metal")
        self.add_class("object", 6, "trash")
        # self.add_class("object", 3, "xyz") #likewise

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset, subset)
        #image_path = os.listdir(self.dataset_dir)
        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        annotations1 = json.load(open(os.path.join(masks_path)))
        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys
        image_path = os.listdir(dataset_dir)
        #print('apelez in')

      #dataset_dir = os.path.join(images_path, subset)
        data = os.listdir(dataset_dir)
       #print(data)
       ##print(dataset_dir)
        images = []
        for i in data:
      
       #image_path = self.imagePaths[i]
        #filename = image_path[i].split(os.path.sep)[-1]
      
      
          k = dataset_dir + '/' + str(i)
          filename = k.split(os.path.sep)[-1]
        #print(filename)
          images.append(filename)

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        #images = load_image(dataset_dir)
        #print(dataset_dir)
        #print(images)
        # Add images
        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects_init = a['filename'] 
            res = [re.findall(r'(\w+?)(\d+)', objects_init)[0] ]
            objects = [res[0][0]]
            print("objects:",objects)
            name_dict = {"cardboard": 1,"glass": 2, "paper":3, "plastic":4, "metal":5, "trash":6} #,"xyz": 3}
            # key = tuple(name_dict)
            num_ids = [name_dict.get(a) for a in objects]
     
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
            #image_path = os.path.join((dataset_dir, #a['filename']
            
                                      #i) for i in image_path)
            
            for i in images:
              if i == a['filename']:

                #print(i)
            #for i in images:
            #k = self.dataset_dir + '/' + str(i)
            #filename = k.split(os.path.sep)[-1]
                p = dataset_dir + "/" + str(i)
                image = skimage.io.imread(p)
                
                height, width = image.shape[:2]

                self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a["filename"],  # use file name as a unique image id
                path = p,
                width=width, height=height,
                polygons=polygons,
                num_ids = num_ids
                )
        #images = []
        #print(images)
  def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Horse/Man dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask , num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

  def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [2]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom(images_path, "train")
    #dataset_train.load_custom(images_path)
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom(images_path, "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=10,
                layers='heads')

In [3]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead








In [4]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [13]:
dataset_train = CustomDataset()
dataset_train.load_custom(images_path, "train")
#dataset_train.load_custom(images_path)
dataset_train.prepare()

Streaming output truncated to the last 5000 lines.
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [

In [14]:
dataset_val = CustomDataset()
dataset_val.load_custom(images_path,  "val")
dataset_val.prepare()

Streaming output truncated to the last 5000 lines.
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [

In [15]:
config = CustomConfig()
config.display()



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                19
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [18]:
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=10,
                layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: /content/drive/MyDrive/Mask_RCNN/logs/object20220320T1635/mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)

/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/usr/local/lib/python3.7/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 1/10
100/100 [==============================] - 336s 3s/step - loss: 1.8109 - rpn_class_loss: 0.0192 - rpn_bbox_loss: 0.4149 - mrcnn_class_loss: 0.1904 - mrcnn_bbox_loss: 0.7045 - mrcnn_mask_loss: 0.4818 - val_loss: 1.3451 - val_rpn_class_loss: 0.0055 - val_rpn_bbox_loss: 0.4568 - val_mrcnn_class_loss: 0.1514 - val_mrcnn_bbox_loss: 0.5305 - val_mrcnn_mask_loss: 0.2009

Epoch 2/10
100/100 [==============================] - 276s 3s/step - loss: 0.9959 - rpn_class_loss: 0.0046 - rpn_bbox_loss: 0.3582 - mrcnn_class_loss: 0.1191 - mrcnn_bbox_loss: 0.3115 - mrcnn_mask_loss: 0.2025 - val_loss: 1.0635 - val_rpn_class_loss: 0.0038 - val_rpn_bbox_loss: 0.4352 - val_mrcnn_class_loss: 0.1304 - val_mrcnn_bbox_loss: 0.2937 - val_mrcnn_mask_loss: 0.2004
Epoch 3/10
100/100 [==============================] - 276s 3s/step - loss: 0.8494 - rpn_class_loss: 0.0067 - rpn_bbox_loss: 0.3764 - mrcnn_class_loss: 0.1015 - mrcnn_bbox_loss: 0.2148 - mrcnn_mask_loss: 0.1501 - val_loss: 0.8839 - val_rpn_clas

ERROR:root:Error processing image {'id': 'trash86.jpg', 'source': 'object', 'path': '/content/drive/MyDrive/trashnet/data_tv/train/trash86.jpg', 'width': 512, 'height': 384, 'polygons': [{'name': 'polygon', 'all_points_x': [348, 353, 242, 178, 134, 98, 106, 138, 162, 163, 178, 230, 210, 202, 227, 242, 252, 227, 204, 166, 140, 174, 194, 258, 310, 318], 'all_points_y': [80, 43, 53, 79, 130, 175, 230, 257, 279, 293, 296, 285, 318, 373, 380, 296, 260, 231, 228, 211, 181, 143, 107, 87, 76, 95]}, {'name': 'polygon', 'all_points_x': [501, 440, 483, 380, 509, 508], 'all_points_y': [102, 213, 300, 375, 379, 99]}], 'num_ids': [6]}
Traceback (most recent call last):
  File "/content/mrcnn/model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/content/mrcnn/model.py", line 1265, in load_image_gt
    class_ids = class_ids[_idx]
IndexError: boolean index did not match indexed array along dimension 0; dimension is 1 but corresponding boolean dimension is 2


 57/100 [================>.............] - ETA: 1:32 - loss: 0.7105 - rpn_class_loss: 0.0038 - rpn_bbox_loss: 0.3268 - mrcnn_class_loss: 0.1058 - mrcnn_bbox_loss: 0.1479 - mrcnn_mask_loss: 0.1261

ERROR:root:Error processing image {'id': 'trash86.jpg', 'source': 'object', 'path': '/content/drive/MyDrive/trashnet/data_tv/train/trash86.jpg', 'width': 512, 'height': 384, 'polygons': [{'name': 'polygon', 'all_points_x': [348, 353, 242, 178, 134, 98, 106, 138, 162, 163, 178, 230, 210, 202, 227, 242, 252, 227, 204, 166, 140, 174, 194, 258, 310, 318], 'all_points_y': [80, 43, 53, 79, 130, 175, 230, 257, 279, 293, 296, 285, 318, 373, 380, 296, 260, 231, 228, 211, 181, 143, 107, 87, 76, 95]}, {'name': 'polygon', 'all_points_x': [501, 440, 483, 380, 509, 508], 'all_points_y': [102, 213, 300, 375, 379, 99]}], 'num_ids': [6]}
Traceback (most recent call last):
  File "/content/mrcnn/model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/content/mrcnn/model.py", line 1265, in load_image_gt
    class_ids = class_ids[_idx]
IndexError: boolean index did not match indexed array along dimension 0; dimension is 1 but corresponding boolean dimension is 2


100/100 [==============================] - 275s 3s/step - loss: 0.7263 - rpn_class_loss: 0.0038 - rpn_bbox_loss: 0.3318 - mrcnn_class_loss: 0.1054 - mrcnn_bbox_loss: 0.1569 - mrcnn_mask_loss: 0.1284 - val_loss: 0.8556 - val_rpn_class_loss: 0.0038 - val_rpn_bbox_loss: 0.4080 - val_mrcnn_class_loss: 0.1134 - val_mrcnn_bbox_loss: 0.1533 - val_mrcnn_mask_loss: 0.1771
Epoch 7/10
100/100 [==============================] - 276s 3s/step - loss: 0.7414 - rpn_class_loss: 0.0051 - rpn_bbox_loss: 0.3258 - mrcnn_class_loss: 0.1265 - mrcnn_bbox_loss: 0.1579 - mrcnn_mask_loss: 0.1261 - val_loss: 0.9283 - val_rpn_class_loss: 0.0049 - val_rpn_bbox_loss: 0.5125 - val_mrcnn_class_loss: 0.0992 - val_mrcnn_bbox_loss: 0.1660 - val_mrcnn_mask_loss: 0.1457
Epoch 8/10
100/100 [==============================] - 276s 3s/step - loss: 0.7507 - rpn_class_loss: 0.0049 - rpn_bbox_loss: 0.3375 - mrcnn_class_loss: 0.1233 - mrcnn_bbox_loss: 0.1619 - mrcnn_mask_loss: 0.1232 - val_loss: 0.7752 - val_rpn_class_loss: 0.0053

In [5]:
class InferenceConfig(CustomConfig):
  GPU_COUNT = 1
  IMAGES_PER_GPU = 1
  DETECTION_MIN_CONFIDENCE = 0.7

In [6]:
config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                19
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [23]:
dataset = CustomDataset()
dataset.load_custom(images_path, "val")
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

Streaming output truncated to the last 5000 lines.
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [1]
objects: ['cardboard']
numids [

In [28]:
MODEL_DIR = '/content/drive/MyDrive/Mask_RCNN/logs'
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
weights_path = model.find_last()
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)


Loading weights  /content/drive/MyDrive/Mask_RCNN/logs/object20220320T1635/mask_rcnn_object_0010.h5
Re-starting from epoch 10


In [29]:
def get_ax(rows=1, cols=1, size=16):
  _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
  return ax

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

def get_ax(rows=1, cols=1, size=16):
  _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
  return ax

image_id = random.choice(dataset.image_ids)
print(image_id)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
  modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id,dataset.image_reference(image_id)))
# Run object detection
results = model.detect([image], verbose=1)
# Display results
ax = get_ax(1)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], dataset.class_names, r['scores'], ax=ax, title="Predictions")
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

# Run object detection
path_to_new_image = '/content/drive/MyDrive/trashnet/predictie/val/cardboard10.jpg'
image1 = mpimg.imread(path_to_new_image)
print(len([image1]))
results1 = model.detect([image1], verbose=1)
# Display results
ax = get_ax(1)
r1 = results1[0]
visualize.display_instances(image1, r1['rois'], r1['masks'], r1['class_ids'],
dataset.class_names, r1['scores'], ax=ax, title="Predictions1")